<a href="https://colab.research.google.com/github/venkateshchettiar/MyProject1/blob/master/Property_Is_Junk_Or_Not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

###**Data Wrangling**

In [2]:
train = pd.read_csv("/content/drive/My Drive/ML/Property_train.csv")
test = pd.read_csv("/content/drive/My Drive/ML/Property_test_share.csv")

In [3]:
train.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,PRIMEUNIT,Channel,Zip,InsurancePremiumIndex,PlotType,Architecture,PriceIndex3,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,0,14224.0,9217.0,10387.0,missing,Direct,21075,623,WHBI,I3Z9,13108.0,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,0,4404.0,2958.0,3400.0,missing,Direct,73129,1689,WHBI,AVYP,3695.0,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,0,8952.0,4568.0,4862.0,missing,Other,29697,2351,W62B,1M5X,7730.0,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,0,7460.0,5361.0,6305.0,missing,Agent,28273,1933,WLQ6,DKOO,6290.0,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,0,9846.0,5428.0,5718.0,missing,Other,29697,482,WHBI,KML6,8583.0,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [4]:
test.head()

,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,PRIMEUNIT,Channel,Zip,InsurancePremiumIndex,PlotType,Architecture,PriceIndex3,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,6LLJ,10645.0,9/22/2010,ZCI3,6015,CAT3,A,missing,9568.0,0,12432.0,6986.0,7133.0,missing,Other,27407,728,WHBI,I3Z9,10860.0,B,5911.0,UA42,BLUE,NC,67549,2006,A,4,8311.0
1,QD0H,9891.0,3/12/2010,RRJQ,7625,OTHER,A,missing,8888.0,0,12173.0,5954.0,6544.0,missing,Agent,32219,1334,32T2,4GZ4,10166.0,A,5370.0,BUNS,BLUE,FL,56570,2004,B,6,7123.0
2,I7GR,8426.0,10/12/2009,ZCI3,6900,CAT2,B,missing,7911.0,0,9134.0,7407.0,7339.0,missing,Other,74135,754,WHBI,TP7R,8500.0,A,6862.0,8YTZ,BLACK,OK,51840,2008,A,1,7994.0
3,5KS5,9269.0,5/20/2010,PRN0,6475,CAT1,B,missing,7773.0,0,8500.0,4241.0,6143.0,missing,Other,80229,1373,WHBI,YIK5,7294.0,A,4872.0,FIVQ,GREEN,CO,83300,2005,A,5,5259.0
4,918O,5366.0,1/13/2009,C8A4,3960,CAT1,A,missing,4134.0,0,5211.0,3352.0,4506.0,missing,Direct,28625,2939,WHBI,YIK5,4120.0,A,3365.0,KT8F,BLACK,NC,95035,2003,A,6,4362.0


In [5]:
a = train.shape
b = test.shape
print(a)
print(b)

(62035, 31)
(10948, 30)


In [6]:
train['data'] = 'train'
test['data'] = 'test'
test['Junk'] = np.nan

In [7]:
train.dtypes

Junk                      int64
InteriorsStyle           object
PriceIndex8              object
ListDate                 object
Material                 object
PriceIndex9               int64
Agency                   object
AreaIncomeType           object
EnvRating                object
PriceIndex7              object
ExpeditedListing          int64
PriceIndex4              object
PriceIndex1              object
PriceIndex6              object
PRIMEUNIT                object
Channel                  object
Zip                       int64
InsurancePremiumIndex     int64
PlotType                 object
Architecture             object
PriceIndex3              object
Region                   object
PriceIndex5              object
SubModel                 object
Facade                   object
State                    object
NormalisedPopulation      int64
BuildYear                 int64
RegionType               object
PropertyAge               int64
PriceIndex2              object
data    

In [8]:
a = train.shape
b = test.shape
print(a)
print(b)

(62035, 32)
(10948, 32)


In [9]:
file = pd.concat([train,test],axis=0)

In [10]:
file.dtypes

Junk                     float64
InteriorsStyle            object
PriceIndex8               object
ListDate                  object
Material                  object
PriceIndex9                int64
Agency                    object
AreaIncomeType            object
EnvRating                 object
PriceIndex7               object
ExpeditedListing           int64
PriceIndex4               object
PriceIndex1               object
PriceIndex6               object
PRIMEUNIT                 object
Channel                   object
Zip                        int64
InsurancePremiumIndex      int64
PlotType                  object
Architecture              object
PriceIndex3               object
Region                    object
PriceIndex5               object
SubModel                  object
Facade                    object
State                     object
NormalisedPopulation       int64
BuildYear                  int64
RegionType                object
PropertyAge                int64
PriceIndex

In [11]:
file.isnull().sum()

Junk                     10948
InteriorsStyle               0
PriceIndex8                  0
ListDate                     0
Material                     0
PriceIndex9                  0
Agency                       0
AreaIncomeType               0
EnvRating                    0
PriceIndex7                  0
ExpeditedListing             0
PriceIndex4                  0
PriceIndex1                  0
PriceIndex6                  0
PRIMEUNIT                    0
Channel                      0
Zip                          0
InsurancePremiumIndex        0
PlotType                     0
Architecture                 0
PriceIndex3                  0
Region                       0
PriceIndex5                  0
SubModel                     0
Facade                       0
State                        0
NormalisedPopulation         0
BuildYear                    0
RegionType                   0
PropertyAge                  0
PriceIndex2                  0
data                         0
dtype: i

In [12]:
file[['ListDate','Zip','SubModel','Architecture','Channel','Facade','EnvRating','PlotType','State']].nunique()

ListDate        517
Zip             153
SubModel        864
Architecture     33
Channel           3
Facade           17
EnvRating         3
PlotType         13
State            37
dtype: int64

In [13]:
file = file.drop(['ListDate','Zip','SubModel'],axis=1)

In [14]:
file.shape

(72983, 29)

In [15]:
file = file.fillna(file.mean())

In [16]:
file.Junk = file.Junk.astype(int)

In [17]:
file.head()

,Junk,InteriorsStyle,PriceIndex8,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,PRIMEUNIT,Channel,InsurancePremiumIndex,PlotType,Architecture,PriceIndex3,Region,PriceIndex5,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2,data
0,0,6LLJ,14674.0,UT7W,8270,CAT3,B,missing,13143.0,0,14224.0,9217.0,10387.0,missing,Direct,623,WHBI,I3Z9,13108.0,A,9022.0,GREEN,MD,42077,2008,A,2,10692.0,train
1,1,XZ9F,4172.0,PRN0,3890,CAT2,B,missing,3461.0,0,4404.0,2958.0,3400.0,missing,Direct,1689,WHBI,AVYP,3695.0,A,2742.0,SILVER,OK,77258,2002,A,7,3615.0,train
2,0,QWEV,8210.0,A9RA,5900,CAT1,B,missing,6922.0,0,8952.0,4568.0,4862.0,missing,Other,2351,W62B,1M5X,7730.0,A,3768.0,SILVER,SC,94514,2004,A,6,5805.0,train
3,0,MK2K,7309.0,RRJQ,7460,OTHER,A,missing,6300.0,0,7460.0,5361.0,6305.0,missing,Agent,1933,WLQ6,DKOO,6290.0,A,5370.0,WHITE,NC,82302,2002,C,7,6444.0,train
4,0,GK4G,9182.0,HE5A,6400,OTHER,B,missing,8330.0,0,9846.0,5428.0,5718.0,missing,Other,482,WHBI,KML6,8583.0,A,4803.0,SILVER,SC,49176,2007,B,3,6286.0,train


In [18]:
cat_cols = file.select_dtypes(include=object).columns[:-1]
cat_cols

Index(['InteriorsStyle', 'PriceIndex8', 'Material', 'Agency', 'AreaIncomeType',
       'EnvRating', 'PriceIndex7', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'PlotType', 'Architecture', 'PriceIndex3',
       'Region', 'PriceIndex5', 'Facade', 'State', 'RegionType',
       'PriceIndex2'],
      dtype='object')

In [19]:
def Create_Dummies(file,cat_cols):
    for col in cat_cols:
        freqs=file[col].value_counts()
        k=freqs.index[freqs>50][:-1]
        for cat in k:
            name=col+'_'+cat
            file[name]=(file[col]==cat).astype(int)
        del file[col]
        print(col)

In [20]:
Create_Dummies(file,cat_cols)

InteriorsStyle
PriceIndex8
Material
Agency
AreaIncomeType
EnvRating
PriceIndex7
PriceIndex4
PriceIndex1
PriceIndex6
PRIMEUNIT
Channel
PlotType
Architecture
PriceIndex3
Region
PriceIndex5
Facade
State
RegionType
PriceIndex2


In [32]:
file.dtypes

Junk                     int64
PriceIndex9              int64
ExpeditedListing         int64
InsurancePremiumIndex    int64
NormalisedPopulation     int64
                         ...  
PriceIndex2_8287.0       int64
PriceIndex2_8151.0       int64
PriceIndex2_8711.0       int64
PriceIndex2_4516.0       int64
PriceIndex2_8168.0       int64
Length: 687, dtype: object

In [40]:
X = file.drop(['Junk','data'],axis=1)
y = file['Junk']

In [41]:
y.dtypes

dtype('int64')

In [42]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [44]:
model = LogisticRegression()

In [45]:
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
prediction = model.predict(X_test)

In [48]:
roc_auc_score(y_test,prediction)

0.5

In [56]:
clf = RandomForestClassifier(verbose=1,n_jobs=-1)
# this here is the base classifier we are going to try
# we will be supplying different parameter ranges to our randomSearchCV which in turn
# will pass it on to this classifier

# Utility function to report best scores. This simply accepts grid scores from 
# our randomSearchCV/GridSearchCV and picks and gives top few combination according to 
# their scores

def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    # above line selects top n grid scores
    # for loop below , prints the rank, score and parameter combination
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")
    
# RandomSearchCV/GridSearchCV accept parameters values as dictionaries.
# In example given below we have constructed dictionary for 
#different parameter values that we want to
# try for randomForest model
param_dist = {"n_estimators":[10,100,500,700],
              "max_depth": [3,5, None],
              "max_features": sp_randint(5, 11),
              "min_samples_split": sp_randint(5, 11),
              "min_samples_leaf": sp_randint(5, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
# run randomized search
n_iter_search = 10
# n_iter parameter of RandomizedSeacrhCV controls, how many 
# parameter combination will be tried; out of all possible given values

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

In [57]:
random_search.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 o

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [59]:
param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": ['auto'],
              "bootstrap": [True, False],
              'class_weight':[None,'balanced'], 
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

In [60]:
n_iter_search = 10

In [61]:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='roc_auc',cv=5)
random_search.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:  2.2min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done 700 out of 700 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:  2.2min finished
[

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [65]:
pred = random_search.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done 700 out of 700 | elapsed:    1.5s finished


In [66]:
roc_auc_score(y_test,pred)

0.6885022987669147

In [67]:
rf=RandomForestClassifier(n_estimators=500,verbose=1,criterion='entropy',min_samples_split=7,
                         bootstrap=False,max_depth=None,max_features=8,min_samples_leaf=5,
                          class_weight="balanced")

In [68]:
rf.fit(X_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   57.3s finished


RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=1, warm_start=False)

In [72]:
preds = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.5s finished


In [73]:
roc_auc_score(y_test,preds)

0.6863841275906618

In [74]:
pd.DataFrame(pred).to_csv("Property.csv",index=False)